In [1]:
import os
import sys
import numpy as np
import pandas as pd
import os.path

data_path="/headnode1/abry4213/data/TS_feature_manuscript/"
sys.path.insert(0, './')
from core_classification_functions import *
from mixed_sigmoid_normalisation import MixedSigmoidScaler
from sklearn.metrics import roc_auc_score,accuracy_score
from copy import deepcopy
# %load_ext rpy2.ipython

In [2]:
# Load data
UCLA_CNP_subjects_to_keep = pd.read_feather(f"{data_path}/time_series_features/UCLA_CNP_filtered_sample_info_catch25_pyspi14.feather")
ABIDE_subjects_to_keep = pd.read_feather(f"{data_path}/time_series_features/ABIDE_filtered_sample_info_catch25_pyspi14.feather")

# Load metadata
UCLA_CNP_metadata = (pd.read_feather(f"{data_path}/input_data/UCLA_CNP_sample_metadata.feather")
                        .assign(Study = "UCLA_CNP")
                        .query("Sample_ID in @UCLA_CNP_subjects_to_keep.Sample_ID"))
ABIDE_metadata = (pd.read_feather(f"{data_path}/input_data/ABIDE_sample_metadata.feather")
                        .assign(Study = "ABIDE")
                        .query("Sample_ID in @ABIDE_subjects_to_keep.Sample_ID"))

# Load univariate time-series feature info
univariate_feature_info = pd.read_csv(f"{data_path}/feature_info/univariate_feature_info.csv")
pairwise_feature_info = pd.read_csv(f"{data_path}/feature_info/pairwise_feature_info.csv")

# Define parameters that you can change
univariate_feature_set = "catch25"
pairwise_feature_set = "pyspi14"

# Load SPI directionality information
SPI_directionality_data = pd.read_csv("SPI_Direction_Info.csv")
SPI_directionality_dict = dict(SPI_directionality_data.values)

# Load univariate time-series feature data for the two datasets
UCLA_CNP_univariate_features = pd.read_feather(f"{data_path}/time_series_features/UCLA_CNP_catch25_filtered.feather")
# ABIDE_univariate_features = pd.read_feather(f"{data_path}/time_series_features/ABIDE_catch25_filtered.feather")

# # Load pyspi14 data for UCLA CNP and ABIDE
# UCLA_CNP_pyspi14 = pd.read_feather(f"{data_path}/time_series_features/UCLA_CNP_pyspi14_filtered.feather")
# ABIDE_pyspi14 = pd.read_feather(f"{data_path}/time_series_features/ABIDE_pyspi14_filtered.feather")


In [3]:
model = svm.SVC(kernel="linear", C=1, class_weight="balanced", probability=True)
pipe = Pipeline([('scaler', MixedSigmoidScaler(unit_variance=True)),
                ('model', model)])
classifier_type = "Linear_SVM_sklearn"

# Define scorers
scorers = [make_scorer(roc_auc_score, needs_proba=True), make_scorer(balanced_accuracy_score), make_scorer(accuracy_score)]
scoring_names = ["AUC", "Balanced_Accuracy", "Accuracy"]

# Classification parameters
num_folds = 10
num_repeats = 10
num_null_iters = 0
num_jobs = 1
RepeatedStratifiedKFold_splitter = RepeatedStratifiedKFold(n_splits=num_folds, n_repeats=num_repeats, random_state=127)

/headnode1/abry4213/.conda/envs/annie_env/lib/python3.9/site-packages/sklearn/metrics/_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


In [4]:
disorder = "ADHD"
dataset_ID = "UCLA_CNP"

class_labels = np.load(f"{data_path}/input_data/{dataset_ID}_{disorder}_class_labels.npy")
sample_IDs = np.load(f"{data_path}/input_data/{dataset_ID}_{disorder}_sample_IDs.npy")

model_name = "UCLA_CNP_ADHD_catch25_feature_SP_Summaries_welch_rect_area_5_1"

In [5]:
print(f"Running {model_name}")
# Define analysis type
if "ROI" in model_name:
    Analysis_Type = "Brain_Region"
elif "combo_catch25_features_all_regions" in model_name:
    Analysis_Type = "Univariate_Combo"
elif "combined_univariate_catch25_and_pyspi14" in model_name:
    Analysis_Type = "SPI_Combo"
elif "catch25_feature" in model_name:
    Analysis_Type = "catch25_feature"
else:
    Analysis_Type = "pyspi14_SPI"

# Find grouping_var
if Analysis_Type=="Brain_Region":
    grouping_var = model_name.split("_ROI_")[1]
elif Analysis_Type=="Univariate_Combo":
    grouping_var = "Combo"
elif Analysis_Type == "SPI_Combo":
    grouping_var = model_name.split("combined_univariate_catch25_and_pyspi14_SPI_")[1]
elif Analysis_Type == "catch25_feature":
    grouping_var = model_name.split("_catch25_feature_")[1]
else:
    grouping_var = model_name.split("_pyspi14_SPI_")[1]

feature_data = np.load(f"{data_path}/time_series_features/processed_numpy_files/{model_name}.npy")


Running UCLA_CNP_ADHD_catch25_feature_SP_Summaries_welch_rect_area_5_1


In [11]:
# Find splits
splits = list(RepeatedStratifiedKFold_splitter.split(feature_data, class_labels))
# Convert splits to a dataframe
splits_df = pd.DataFrame(splits, columns = ["Train", "Test"])

# Assign the fold and repeat numbers
splits_df["Fold"] = splits_df.index % num_folds
splits_df["Repeat"] = splits_df.index // num_repeats

fold_res_list = []
class_1_col_list = []

# Iterate over each row of fold_splits dataframe
for i, row in splits_df.iterrows():
    fold_num = row["Fold"]
    repeat_num = row["Repeat"]
    train_indices = row["Train"]
    test_indices = row["Test"]

    train_data = feature_data[train_indices]
    test_data = feature_data[test_indices]

    train_data = feature_data[train_indices]
    test_data = feature_data[test_indices]

    train_labels = class_labels[train_indices]
    test_labels = class_labels[test_indices]

    # Fit pipe to train_data
    loop_pipe = deepcopy(pipe)
    loop_pipe.fit(train_data, train_labels)
    test_preds = loop_pipe.predict(test_data)
    test_preds_prob = loop_pipe.predict_proba(test_data)

    # Find out which column corresponds to which class
    fitted_model_classes = loop_pipe.classes_
    class_1_col = np.where(fitted_model_classes==1)[0][0]
    test_preds_prob_data = test_preds_prob[:,class_1_col]
    class_1_col_list.append(class_1_col)

    # # Figure out which column to keep: 
    # if len(test_preds[test_preds==0]) == 0:
    #     # Find whichever column of test_preds_prob has the lower mean
    #     prob_col = np.argmin([np.mean(test_preds_prob[:,0]), np.mean(test_preds_prob[:,1])])
    # elif len(test_preds[test_preds==1]) == 0:
    #     # Find whichever column of test_preds_prob has the higher mean
    #     prob_col = np.argmax([np.mean(test_preds_prob[:,0]), np.mean(test_preds_prob[:,1])])
    # elif np.mean(test_preds_prob[test_preds==0]) < 0.5: 
    #     prob_col = 1
    # else: 
    #     prob_col = 0
    # test_preds_prob_data = test_preds_prob[:,prob_col]

    # Compute AUC
    auc = roc_auc_score(test_labels, test_preds_prob_data)

    this_loop_res = pd.DataFrame({"Fold": fold_num, "Repeat": repeat_num, "AUC": auc}, index=[0])
    fold_res_list.append(this_loop_res)

all_AUC_fold_res = pd.concat(fold_res_list)